<a href="https://colab.research.google.com/github/jonasvdbran/Masterthesis-SCRPPP/blob/main/Local_Search_Housekeeping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we import the container_classes file. **Upload container_classes.py via the next block of code.**

In [7]:
from google.colab import files
uploaded = files.upload()

Saving container_classes.py to container_classes (1).py


In [8]:
from container_classes import *

We again will test the algorithm on our example bay.

In [9]:
module=Module([Bay([Stack([Container(1),Container(2)],4),
    Stack([Container(3),Container(3),Container(4),Container(1)],4),Stack([Container(2)],4),Stack([Container(1),Container(4),Container(4),Container(5)],4)],4)],4)
print(module)

Bay 0:
  | 1 |   | 5
  | 4 |   | 4
2 | 3 |   | 4
1 | 3 | 2 | 1


Note that all functions in this python notebook also work on bay objects, not only on module objects. This extra functionality isn't required and can therefore be omitted if wanted (as a bay can be seen as type of module with 1 bay).

# Estimating shifting costs

The local search heuristic algorithm estimates the shifting moves by categorizing the containers via algorithm 6.

In [10]:
def estimate_moves_stack(stack: Stack,max_smallest_values: int,min_biggest_values:int) -> float:
  """

  Args:
    stack: stack with containers
    max_smallest_values: maximum of the smallest values of all stacks
    min_biggest_values: minimum of the biggest values of all stacks

  Returns: Estimated amount of shifting moves for stack

  """
  if stack.is_empty():
    return 0

  score=0
  container_values=[container.value for container in stack.containers]
  smallest_under_array=get_smallest_under(container_values)

  # Give a score to containers
  for i,value in enumerate(container_values):
    # Categorize:
    if i==0 or value==np.inf:
      # If we are at bottom: 0 score, same for immovable containers (np.inf)
      continue
    smallest_under=smallest_under_array[i-1] #Excluding itself
    # No relocations needed
    if smallest_under>value:
      score+=0
    # Possible relocations but not sure
    elif smallest_under==value:
      score+=0.5
    # Relocations needed and good stack exists
    elif smallest_under>min_biggest_values or value < max_smallest_values:
      score+=1
    # Relocations needed but no good stack exists
    else:
      score+=1.4
  return score

In [11]:
def estimate_relocation_moves(container_structure: Bay | Module, max_smallest_values=None, min_biggest_values=None):
  """

  Args:
    container_structure: Bay or Module for which you want to estimate its relocation moves
    neighbours: amount of neighbouring bays a container in the way of the current container can be moved to
    max_smallest_values: internal variable, maximum of the smallest values of all stacks
    min_biggest_values: internal variable, minimum of the biggest values of all stacks

  Returns: estimate of the amount of relocation moves for container_structure

  """
  score=0
  # Module
  if isinstance(container_structure,Module):
    smallest_values = [min(stack).value if not stack.is_empty() else -1 for bay in container_structure for stack in bay]
    biggest_values = [max(stack).value if not stack.is_empty() else -1 for bay in container_structure for stack in bay]

    # Find relocation moves per bay and add them together
    for bay in container_structure:
      score+=estimate_relocation_moves(bay,max(smallest_values),min(biggest_values))
    return score

  # Bay
  else:
    if max_smallest_values == None:
      smallest_values = [min(stack) if not stack.is_empty() else -1 for stack in container_structure]
      biggest_values = [max(stack) if not stack.is_empty() else -1 for stack in container_structure]
      max_smallest_values=max(smallest_values).value
      min_biggest_values=min(biggest_values).value

    # Find relocation moves per stack and add them together
    for stack in container_structure:
      score+=estimate_moves_stack(stack,max_smallest_values,min_biggest_values)

    return score

# Find correct stacks

The next functions determine ideal stacks and specify to which stack a container will be moved, according to the local search algorithm, algorithm 3.

In [21]:
def get_smallest_under(container_values: list)-> np.array:
  """

  Args:
    container_values: list of container values

  Returns: array of smallest values under
  """
  return np.minimum.accumulate(np.array(container_values))

In [22]:
def get_to_move(container_values: list,threshold: int)-> np.array:
  """

  Args:
    container_values: list of container values
    threshold: threshold value

  Returns: array of container values that need to be moved in destination stack

  """
  smallest_under=get_smallest_under(container_values)
  # Find containers that need to be moved when placing a container w value threshold in stack (i.e. the smallest value under it is smaller than threshold)
  return np.array(container_values)[smallest_under<threshold]

In [13]:
def get_ideal_stacks_list(container_structure : Bay | Module, container_index: tuple,
                            container_value: int,alpha: float,excluded_stacks:list,
                             neighbours:int, empty_min_value:int,bay_index=None, max_smallest_values=None, min_biggest_values=None) -> list:
  """

  Args:

    container_structure: Bay or Module
    container_index: index of current container in container_structure
    container_value: value of current container
    alpha: cost of pre-processing move (w.r.t. relocation move)
    excluded_stacks: list with excluded stacks (from which it is not allowed to pick)
    neighbours: amount of neighbouring bays a container in the way of the current container can be moved to
    empty_min_value: min value of an empty stack
    bay_index: internal variable, index of current bay in module

  Returns: list with ideal stacks, represented as tuples (stack_index,stack)

  """
  ideal_stacks=[]
  # Module

  # Only searches for ideal stacks in range of neighbours_containers_in_way
  if isinstance(container_structure,Module):
    current_bay_index=container_index[0]
    start_index = max(0, current_bay_index - neighbours)
    end_index = min(len(container_structure), current_bay_index+neighbours+1)
    selected_bay_indices=range(start_index, end_index)


    smallest_values=[min(stack) for bay in container_structure for stack in bay if not stack.is_empty()]
    biggest_values=[max(stack) for bay in container_structure for stack in bay if not stack.is_empty()]

    # Finds ideal stacks to place container in, bay per bay
    for bay_index, bay in ((i, container_structure[i]) for i in selected_bay_indices):
        ideal_stacks_list=get_ideal_stacks_list(bay, container_index, container_value, alpha,
                                                    excluded_stacks,neighbours,empty_min_value, bay_index,
                                                    max(smallest_values).value,min(biggest_values).value)
        ideal_stacks += ideal_stacks_list
    return ideal_stacks
  # Bay
  else:
    # Loops over all stacks in bay to determine ideal stacks
    for stack_index,stack in enumerate(container_structure):
      stack_index=(bay_index,stack_index) if isinstance(bay_index, int) else stack_index

      # Stack is full -> no container can be added so not a ideal stack
      if stack.is_full():
        continue

      # Can't move to excluded stacks
      if stack_index in excluded_stacks:
        continue
      # Gives a fixed value as minimum for empty stacks
      if stack.is_empty():
        min_value= empty_min_value
      else:
        min_value= min(stack).value

      # If max_smallest_values is specified, this means we are in a module. Otherwise calculate it for a bay.
      if max_smallest_values == None:
        smallest_values=[min(stack) for stack in container_structure if not stack.is_empty()]
        biggest_values=[max(stack) for stack in container_structure if not stack.is_empty()]
        max_smallest_values=max(smallest_values).value
        min_biggest_values=min(biggest_values).value


      estimate_relocation_moves_stack=estimate_moves_stack(stack,max_smallest_values,min_biggest_values)
      # Determine if stack is ideal via criterium
      if min_value>container_value and estimate_relocation_moves_stack<= alpha:
        ideal_stacks.append((stack_index,min_value)) #Also stores minimum value

    return ideal_stacks

In [14]:
def get_smallest_value_stacks(container_structure : Bay | Module,container_index:int, neighbours:int,excluded_stacks:list,empty_min_value:int,bay_index=None):
  """

  Args:
    container_structure: Bay or Module
    excluded_stacks: list with excluded stacks (from which it is not allowed to pick)
    empty_min_value: min value of an empty stack
    bay_index: internal variable, index of current bay in module

  Returns: list with smallest value from stacks, represented as tuples (stack_index, minimum of stack)

  """

  smallest_value_stacks=[]

  # Only finds minimum values of stacks in range of neighbours_containers_in_way
  if isinstance(container_structure,Module):
    current_bay_index=container_index[0]
    start_index = max(0, current_bay_index - neighbours)
    end_index = min(len(container_structure), current_bay_index+neighbours+1)
    selected_bay_indices=range(start_index, end_index)

    # Find the smallest values for stacks, bay per bay
    for bay_index, bay in ((i, container_structure[i]) for i in selected_bay_indices):
       smallest_value_stacks += get_smallest_value_stacks(bay,container_index,neighbours,excluded_stacks,empty_min_value,bay_index)
    return smallest_value_stacks

  else:

    # Find smallest value for stacks in bay
    for stack_index,stack in enumerate(container_structure):
      stack_index=(bay_index,stack_index) if isinstance(bay_index, int) else stack_index
      if stack_index in excluded_stacks:
        continue
      if stack.is_empty():
        smallest_value_stacks.append((stack_index,empty_min_value))
      elif stack.is_full():
        continue
      else:
        # Find smallest value, and stores stack_index
        smallest_value_stacks.append((stack_index,min(stack).value))
    return smallest_value_stacks

In [15]:
def get_stack_to_move_to(container_structure : Bay | Module, container_index: tuple,
                            container_value: int,alpha: float,lambda1: int, lambda2: int,
                            excluded_stacks:list, neighbours:int, empty_min_value:int):
  """

  Args:
    container_structure: Bay or Module
    container_index: index of current container in container_structure
    container_value: value of current container
    alpha: cost of pre-processing move (w.r.t. relocation move)
    lambda1: number of stacks to consider for selection based on their highest value in the correct_stacks list
    lambda2: number of smallest value stacks to consider for random selection based on their highest value, when no correct stacks are found in neighbouring bays
    excluded_stacks:list with excluded stacks (from which it is not allowed to pick)
    neighbours: amount of neighbouring bays a container in the way of the current container can be moved to

  Returns:

  """
  # Gets ideal stacks to move container to
  ideal_stacks=get_ideal_stacks_list(container_structure,container_index,container_value,alpha,excluded_stacks,neighbours,empty_min_value)
  if len(ideal_stacks)>0:
    # Lambda1=1 -> get ideal stack with minimum value
    if lambda1==1:
      return min(ideal_stacks, key=lambda x: x[1])[0]
    sorted_stacks = sorted(ideal_stacks, key=lambda x: x[1])

    # Select the `lambda1` stacks with the smallest values
    smallest_stacks = sorted_stacks[:lambda1]
    return random.choice(smallest_stacks)[0]

  #If no ideal stacks:
  else:
    smallest_values_stacks= get_smallest_value_stacks(container_structure,container_index,neighbours,excluded_stacks,empty_min_value)
    if smallest_values_stacks==[]:
      return False # No stack to put container in
    # Get the ones with smallest value in stack
    if lambda2==1:
      return min(smallest_values_stacks, key=lambda x: x[1])[0]
    else:
      sorted_stacks=sorted(smallest_values_stacks, key=lambda x: x[1])
      smallest_stacks= sorted_stacks[:lambda2]
      return random.choice(smallest_stacks)[0]

# Move to different stack

In this section, algorithm 4 is implemented, as part of the local search housekeeping algorithm.

In [16]:
def move_to_different_stack(container_structure : Bay | Module, current_container_index:tuple,destination_stack_index:tuple,
                            alpha:float,lambda1:int, lambda2:int,neighbours:int,empty_min_value):
  """

  Args:
    container_structure: Bay or Module
    current_container_index: index of container you want to move
    destination_stack_index: index of destination stack
    alpha: cost of pre-processing move (w.r.t. relocation move)
    lambda1: lambda1: number of stacks to consider for selection based on their highest value in the correct_stacks list
    lambda2: number of smallest value stacks to consider for random selection based on their highest value, when no correct stacks are found in neighbouring bays
    neighbours: amount of neighbouring bays a container in the way of the current container can be moved to
    empty_min_value: min value of an empty stack


  Returns: container_structure, moves where the current container was moved from current_container_index to destination_stack_index

  """
  new_container_structure=container_structure.copy()

  current_stack_index=current_container_index[:-1] if isinstance(new_container_structure,Module) else current_container_index[0]
  current_level=current_container_index[-1]

  current_stack=new_container_structure[current_stack_index]
  destination_stack=new_container_structure[destination_stack_index]


  value=current_stack[current_level].value

  #Get the containers on top of container we want to move and the ones in destination stack we need to move
  on_top= list(current_stack[current_level+1:])
  destination_stack_to_move= list(get_to_move([container.value for container in destination_stack.containers],value))

  moves=[]
  # While we have containers to move, move them out of the way
  while len(on_top)>0 or len(destination_stack_to_move)>0:

    # If on_top is empty, give it value -1
    if len(on_top)==0:
      top_on_top=-1
    else:
      top_on_top=on_top[-1]

    # Same for destination_stack
    if len(destination_stack_to_move)==0:
      top_destination_stack_to_move=-1
    else:
      top_destination_stack_to_move=destination_stack_to_move[-1]

    # Pick the one with highest value to move first
    if top_on_top>=top_destination_stack_to_move:
      container_value=on_top.pop(-1)
      stack_index=current_stack_index
    else:
      container_value=destination_stack_to_move.pop(-1)
      stack_index=destination_stack_index
    # Get best stack to move this container to
    stack_to_move_to= get_stack_to_move_to(new_container_structure, current_container_index,
                            container_value,alpha,lambda1, lambda2,[current_stack_index,destination_stack_index], neighbours,empty_min_value)
    if stack_to_move_to==False:
      return container_structure,moves
    # Perform the move
    new_container_structure.move(stack_index,stack_to_move_to)
    stack_IDs= [new_container_structure[stack_index].ID,new_container_structure[stack_to_move_to].ID]
    # If stacks have IDs use that, otherwise use index
    if stack_IDs[0] and stack_IDs[1]:
      moves.append((stack_IDs[0],stack_IDs[1]))
    else:
      moves.append((stack_index,stack_to_move_to))


  # If there isn't any space freed up in destination stack, just return and don't move the container
  if destination_stack.is_full():
    return container_structure,moves

  # Put current container in destination stack
  container=current_stack.pop()
  container.placed=True
  destination_stack.push(container)

  stack_IDs= [new_container_structure[current_stack_index].ID,new_container_structure[destination_stack_index].ID]
  # If stacks have IDs use that, otherwise use index
  if stack_IDs[0] and stack_IDs[1]:
      moves.append((stack_IDs[0],stack_IDs[1]))
  else:
    moves.append((current_stack_index,destination_stack_index))

  return new_container_structure,moves

# Move to current stack

In this section algorithm 5 is implemented, as part of the local search housekeeping algorithm.

In [17]:
def move_to_current_stack(container_structure : Bay | Module, current_container_index:tuple,
                            alpha:float,lambda1:int, lambda2:int,neighbours:int,empty_min_value:int):
  """

  Args:
    container_structure: Bay or Module
    current_container_index: index of container you want to move
    alpha: cost of pre-processing move (w.r.t. relocation move)
    lambda1: number of stacks to consider for selection based on their highest value in the correct_stacks list
    lambda2: number of smallest value stacks to consider for random selection based on their highest value, when no correct stacks are found in neighbouring bays
    neighbours: amount of neighbouring bays a container in the way of the current container can be moved to
    empty_min_value: min value of an empty stack

  Returns: current_stack_index, moves where the current container was moved from current_container_index to current_stack_index

  """
  new_container_structure=container_structure.copy()

  current_stack_index=current_container_index[:-1] if isinstance(new_container_structure,Module) else current_container_index[0]
  current_bay_index=current_container_index[0]
  current_level=current_container_index[-1]

  current_stack=new_container_structure[current_stack_index]

  if isinstance(new_container_structure, Module):
    current_bay=new_container_structure[current_bay_index]
  else:
    current_bay=new_container_structure

  value=current_stack[current_level].value
  # Get the ones on top of the one we need to move, we will need to move these first
  on_top= list(current_stack[current_level+1:])
  moves=[]

  # Move the ones blocking the container
  while len(on_top)>0:
    container_value=on_top.pop(-1)

    # Get correct stack to move container to
    stack_to_move_to=get_stack_to_move_to(new_container_structure, current_container_index,
                            container_value,alpha,lambda1, lambda2,[current_stack_index], neighbours,empty_min_value)
    if stack_to_move_to==False:
      return container_structure,moves
    # Put it inside this stack
    new_container_structure.move(current_stack_index,stack_to_move_to)

    stack_IDs= [new_container_structure[current_stack_index].ID,new_container_structure[stack_to_move_to].ID]
    # If stacks have IDs use that, otherwise use index
    if stack_IDs[0] and stack_IDs[1]:
      moves.append((stack_IDs[0],stack_IDs[1]))
    else:
      moves.append((current_stack_index,stack_to_move_to))


  # Put container temporarily aside in highest stack in bay with space available
  if isinstance(new_container_structure,Module):
      current_bay_index=current_container_index[0]
      start_index = max(0, current_bay_index - neighbours)
      end_index = min(len(new_container_structure), current_bay_index+neighbours+1)
  else:
      start_index=0
      end_index=1

  valid_stacks = [(bay_index, stack_index) if isinstance(new_container_structure, Module) else stack_index
    for bay_index in range(start_index, end_index)
    for stack_index, stack in enumerate(new_container_structure[bay_index] if isinstance(new_container_structure, Module) else new_container_structure)
    if (bay_index, stack_index) != current_stack_index and not stack.is_full()]

  put_aside_stack_index = max(valid_stacks, key=lambda idx: len(new_container_structure[idx]), default=None)




  if put_aside_stack_index==None:
    return container_structure,moves
  # Put current container aside
  new_container_structure.move(current_stack_index,put_aside_stack_index)
  stack_IDs= [new_container_structure[current_stack_index].ID,new_container_structure[put_aside_stack_index].ID]
  if stack_IDs[0] and stack_IDs[1]:
      moves.append((stack_IDs[0],stack_IDs[1]))
  else:
     moves.append((current_stack_index,put_aside_stack_index))




  # Get containers under current_container that need to be moved to place current container correctly
  to_move_under_container= list(get_to_move(current_stack[:current_level],value))
  while len(to_move_under_container)>0:
    container_value=to_move_under_container.pop(-1)

    stack_to_move_to=get_stack_to_move_to(new_container_structure, current_container_index,
                            container_value,alpha,lambda1, lambda2,[current_stack_index,put_aside_stack_index], neighbours,empty_min_value)
    if stack_to_move_to==False:
      return container_structure,moves
    # Perform the move
    new_container_structure.move(current_stack_index,stack_to_move_to)
    stack_IDs= [new_container_structure[current_stack_index].ID,new_container_structure[stack_to_move_to].ID]
    # If stacks have IDs use that, otherwise use index
    if stack_IDs[0] and stack_IDs[1]:
      moves.append((stack_IDs[0],stack_IDs[1]))
    else:
      moves.append((current_stack_index,stack_to_move_to))

  # Move current container back in current stack

  container=new_container_structure[put_aside_stack_index].pop()
  container.placed=True
  current_stack.push(container)

  stack_IDs= [new_container_structure[put_aside_stack_index].ID,new_container_structure[current_stack_index].ID]
  # If stacks have IDs use that, otherwise use index
  if stack_IDs[0] and stack_IDs[1]:
    moves.append((stack_IDs[0],stack_IDs[1]))
  else:
    moves.append((put_aside_stack_index,current_stack_index))

  return new_container_structure,moves

# Housekeeping modules

Finally, we have the local search algorithm, algorithm 3.

In [44]:
def housekeeping(container_structure : Bay | Module, alpha: float, lambda1=1,
                   lambda2=1, neighbours=False,neighbours_containers_in_way=False,empty_min_value=-1,estimate_shifting_function=estimate_relocation_moves) -> Bay | Module:
  """

  Args:
    container_structure: Bay or Module you want to pre-proces
    alpha: cost of pre-processing move (w.r.t. relocation move)
    lambda1: number of stacks to consider for selection based on their highest value in the correct_stacks list
    lambda2: number of smallest value stacks to consider for random selection based on their highest value, when no correct stacks are found in neighbouring bays
    neighbours: amount of neighbouring bays the current container can be moved to
    neighbours_containers_in_way: amount of bays a container in the way of the current container can be moved to
    empty_min_value: minimum value of an empty stack

  Returns: pre-processed Bay/Module

  """
  # If we have no neigbours specified, we consider the whole module
  if isinstance(container_structure,Module) and not neighbours:
    neighbours=len(container_structure)

  # Copy the container structure, we want to make changes to it but not sure if we will keep them
  container_structure=container_structure.copy()
  housekeeping_moves=[]

  # Calculate reloc. moves to later calculate improvement with.
  expected_number_relocation_moves_before=estimate_shifting_function(container_structure)

   # Find the maximum value to start our loop with
  max_value= container_structure.max_container().value

   # Loops over all containers. Starts with containers with highest value.
  for t in range(max_value,1,-1):
    containers_with_time_t = container_structure.where_value(t) #List with tuples containing the positions where the value t is situated in bay
    # Goes over all the containers with time t and places them.
    while len(containers_with_time_t)>0:
      # A container is a tuple, containing the place of the container
      inspected_container=containers_with_time_t.pop(0)
      current_stack_index=inspected_container[:-1] if isinstance(container_structure,Module) else inspected_container[0]

      #Starting values to find best improvement after each iteration
      best_improvement=0
      best_moves=[]
      best_container_structure=container_structure
      #Store relocation moves before moving to calculate improvement after moving
      best_reloc_moves=expected_number_relocation_moves_before

      # Determine neigbouring bays to loop over
      if isinstance(container_structure,Module):
        current_bay_index=inspected_container[0]
        start_index = max(0, current_bay_index - neighbours)
        end_index = min(len(container_structure), current_bay_index+neighbours+1)
      else:
        start_index=0
        end_index=1

      # Loop over all bays
      for bay_index in range(start_index,end_index):
        # If we have a module it gets its bays, if it's a bay we get the bay itself
        bay=container_structure[bay_index] if isinstance(container_structure,Module) else container_structure
        for stack_index in range(len(bay)):
          target_location = (bay_index, stack_index) if isinstance(container_structure, Module) else stack_index

           # Place container in a stack to see how much it improves our situation
          if target_location!=current_stack_index:
            new_container_structure,moves=move_to_different_stack(container_structure,inspected_container,target_location,alpha,
                                                                  lambda1, lambda2,neighbours_containers_in_way,empty_min_value)
          # " " "  " in same stack  " " " "
          else:
            new_container_structure,moves=move_to_current_stack(container_structure,inspected_container,alpha,lambda1, lambda2,
                                                                neighbours_containers_in_way, empty_min_value)

          # Calculate relocation moves after placing the container
          # -> get f(B_s)
          expected_number_relocation_moves_after=estimate_shifting_function(new_container_structure)
          #Find improvement
          # I(s)= f(B)-(alpha*m(s)+f(B_s))
          improvement=expected_number_relocation_moves_before-(alpha*len(moves)+expected_number_relocation_moves_after)

          print(f"improvement {stack_index}: {improvement}")
          print(f'moves: {len(moves)}')
          print(f'expected_number_relocation_moves_before:{expected_number_relocation_moves_before}')
          print(f'expected_number_relocation_moves_after:{expected_number_relocation_moves_after}')
          print(new_container_structure)
          # If better than best previous improvement: store it
          if improvement>0 and improvement>best_improvement:
            best_improvement=improvement
            best_moves=moves
            best_container_structure=new_container_structure
            best_reloc_moves=expected_number_relocation_moves_after


          else:
            continue

      #print(best_container_structure)
      expected_number_relocation_moves_before=best_reloc_moves

      # Update container_structure and get (possible) new locations of containers w value t
      if container_structure!=best_container_structure:
        housekeeping_moves.append(best_moves)
        container_structure=best_container_structure
        containers_with_time_t = container_structure.where_value(t)

      # If no moves are good -> continue with next container
      else:
        continue

  flattened_housekeeping_moves = [from_to_pair for sublist in housekeeping_moves for from_to_pair in sublist]

  return container_structure,flattened_housekeeping_moves

# Performing algorithm on the bay

In [30]:
alpha=0.6

In [5]:
print(module)

Bay 0:
  | 1 |   | 5
  | 4 |   | 4
2 | 3 |   | 4
1 | 3 | 2 | 1


In [23]:
estimate_relocation_moves(module)

7.1

In [31]:
housekeeped_module,moves=housekeeping(module,alpha,2,2)

improvement 0: 1.2999999999999998
moves: 3
expected_number_relocation_moves_before:7.1
expected_number_relocation_moves_after:4.0
Bay 0:
  | 1 |   |  
  | 4 | 1 | 4
  | 3 | 2 | 4
5 | 3 | 2 | 1
improvement 1: -1.9000000000000004
moves: 5
expected_number_relocation_moves_before:7.1
expected_number_relocation_moves_after:6
Bay 0:
  |   | 3 |  
4 |   | 3 | 4
2 |   | 1 | 4
1 | 5 | 2 | 1
improvement 2: 0.39999999999999947
moves: 2
expected_number_relocation_moves_before:7.1
expected_number_relocation_moves_after:5.5
Bay 0:
  | 1 |   |  
2 | 4 |   | 4
2 | 3 |   | 4
1 | 3 | 5 | 1
improvement 3: -0.40000000000000036
moves: 5
expected_number_relocation_moves_before:7.1
expected_number_relocation_moves_after:4.5
Bay 0:
  | 1 | 1 |  
  | 4 | 4 |  
2 | 3 | 4 |  
1 | 3 | 2 | 5
improvement 0: -1.5
moves: 2
expected_number_relocation_moves_before:4.0
expected_number_relocation_moves_after:4.3
Bay 0:
  |   | 1 |  
  |   | 1 | 4
4 | 3 | 2 | 4
5 | 3 | 2 | 1
improvement 1: -4.300000000000001
moves: 5
expe

In [32]:
print(housekeeped_module)

Bay 0:
  | 1 |   |  
  | 4 | 1 | 4
  | 3 | 2 | 4
5 | 3 | 2 | 1


In [33]:
estimate_relocation_moves(housekeeped_module)

4.0

In [34]:
SC,d=CalculateShiftingCosts(housekeeped_module,k_max=4)

In [35]:
sum(SC.values())

4.0

In [36]:
print(module)

Bay 0:
  | 1 |   | 5
  | 4 |   | 4
2 | 3 |   | 4
1 | 3 | 2 | 1


In [39]:
improvement=estimate_relocation_moves(module)-estimate_relocation_moves(housekeeped_module)-alpha*len(moves)
print(improvement)

1.2999999999999998


# Using the look-ahead shifting costs to calculate improvement

As mentioned in section 4.5.1, we can also use the look-ahead shifting costs in the housekeeping algorithm instead of the category based algorithm.

In [40]:
def look_ahead_shifting_costs(module):
  return sum(CalculateShiftingCosts(module)[0].values())

In [42]:
look_ahead_shifting_costs(module)

7.25

In [45]:
new_module,moves=housekeeping(module,alpha,2,2,estimate_shifting_function=look_ahead_shifting_costs)

improvement 0: 1.4500000000000002
moves: 3
expected_number_relocation_moves_before:7.25
expected_number_relocation_moves_after:4.0
Bay 0:
  | 1 |   |  
  | 4 | 1 | 4
  | 3 | 2 | 4
5 | 3 | 2 | 1
improvement 1: -1.75
moves: 5
expected_number_relocation_moves_before:7.25
expected_number_relocation_moves_after:6.0
Bay 0:
  |   | 3 |  
3 |   | 4 | 4
2 |   | 1 | 4
1 | 5 | 2 | 1
improvement 2: 0.5499999999999998
moves: 2
expected_number_relocation_moves_before:7.25
expected_number_relocation_moves_after:5.5
Bay 0:
  | 1 |   |  
2 | 4 |   | 4
2 | 3 |   | 4
1 | 3 | 5 | 1
improvement 3: -0.25
moves: 5
expected_number_relocation_moves_before:7.25
expected_number_relocation_moves_after:4.5
Bay 0:
  | 1 | 1 |  
  | 4 | 4 |  
2 | 3 | 4 |  
1 | 3 | 2 | 5
improvement 0: -1.3666666666666671
moves: 2
expected_number_relocation_moves_before:4.0
expected_number_relocation_moves_after:4.166666666666667
Bay 0:
  |   |   | 1
  |   | 1 | 4
4 | 3 | 2 | 4
5 | 3 | 2 | 1
improvement 1: -4.0
moves: 5
expected_numb

This results in the bay found with the other housekeeping algorithm, and leads to a better total improvement.

In [46]:
print(new_module)

Bay 0:
  | 1 |   |  
4 | 4 | 1 |  
4 | 3 | 2 |  
5 | 3 | 2 | 1


In [47]:
look_ahead_shifting_costs(new_module)

2.5

In [48]:
Improvement=look_ahead_shifting_costs(module)-look_ahead_shifting_costs(new_module)-alpha*len(moves)
print(Improvement)

1.75
